In [84]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import geocoder
from geopy.geocoders import Nominatim 
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [67]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url).text

# parse the page, locate HTML tags with relevant information
soup = BeautifulSoup(response, 'lxml')

In [71]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [72]:
print(df.shape)

(103, 3)


In [74]:
def get_coords(postcode):
    return geocoder.arcgis(f'{postcode}, Toronto, Ontario').latlng

In [75]:
postcodes = df['PostalCode'].tolist()
coords = [get_coords(code) for code in postcodes] # apply function
print("Coordinates are obtained.")

Coordinates are obtained.


In [77]:
# a new dataframe to store coordinates
coords_arcgis = df.copy()

# add columns for latitudes and longitudes
coords_arcgis['Latitude'] = [coord[0] for coord in coords]
coords_arcgis['Longitude'] = [coord[1] for coord in coords]
coords_arcgis.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [79]:
# load the coordinates file
coords_google = pd.read_csv('Geospatial_Coordinates.csv',
                            header=0,
                            names=['PostalCode', 'Latitude', 'Longitude']
                           )

# merge with the original dataframe
toronto = df.merge(coords_google, on='PostalCode', how='inner')
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [81]:
toronto.to_csv('Toronto-Pt2.csv', index=False)

<h1>Part 3</h1>

<h2>Visualization: Toronto neighborhoods</h2>

In [87]:
# get the coordinates of Toronto using Nominatim geocoder
geolocator = Nominatim(user_agent='tor_explorer')
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
print(f"The geograpical coordinates of Toronto are {latitude}, {longitude}.")

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [89]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, nbhd in zip(toronto['Latitude'],
                                   toronto['Longitude'],
                                   toronto['Borough'],
                                   toronto['Neighborhood']):
    label = f"{nbhd}, {borough}"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='purple',
                        fill=True,
                        fill_color='pink',
                        fill_opacity=0.7,
                        parse_html=False
                       ).add_to(map_tor)

map_tor

<h1>Select boroughs of interest</h1>

Borough names with the word 'Toronto' in them will be explored further.

In [92]:
tor_boi = toronto[toronto['Borough'].str.contains('Toronto')]
tor_boi.reset_index(drop=True, inplace=True)
tor_boi.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [93]:
map_tor_boi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, nbhd in zip(tor_boi['Latitude'],
                                   tor_boi['Longitude'],
                                   tor_boi['Borough'],
                                   tor_boi['Neighborhood']):
    label = folium.Popup(f"{nbhd}, {borough}", parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='purple',
                        fill=True,
                        fill_color='pink',
                        fill_opacity=0.7,
                        parse_html=False
                       ).add_to(map_tor_boi)
map_tor_boi